In [23]:
#loading libraries
import pandas as pd, numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [24]:
#Loading dataset
import os

Shopping = pd.read_csv(open(os.path.join("C://Users//hp//Desktop//dma cp", "sh_binary.csv"), 'r', encoding='utf-8'), index_col=0)

In [26]:
df5=Shopping

In [28]:
df5.index=[ x for x in range (0,len(df5.values))]

In [29]:
df5.head(1)

,b_id,b_name,text,u_id,service,place,price,product
0,jaJnPIX9VxsFyfV5zQwk0g,Coca-Cola Store Las Vegas,Lots of people on the second floor,29yBaTFMSh-l6wNwy9d_mA,1,0,0,0


In [30]:
import os
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [32]:
categories =['service','price','place','product']
categories

['service', 'price', 'place', 'product']

In [34]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import re
import sys
import warnings
data = df5
if not sys.warnoptions:
    warnings.simplefilter("ignore")
def cleanHtml(sentence):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', str(sentence))
    return cleantext
def cleanPunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    cleaned = cleaned.strip()
    cleaned = cleaned.replace("\n"," ")
    return cleaned
def keepAlpha(sentence):
    alpha_sent = ""
    for word in sentence.split():
        alpha_word = re.sub('[^a-z A-Z]+', ' ', word)
        alpha_sent += alpha_word
        alpha_sent += " "
    alpha_sent = alpha_sent.strip()
    return alpha_sent
data['text'] = data['text'].str.lower()
data['text'] = data['text'].apply(cleanHtml)
data['text'] = data['text'].apply(cleanPunc)
data['text'] = data['text'].apply(keepAlpha)

In [35]:
stemmer = SnowballStemmer("english")
def stemming(sentence):
    stemSentence = ""
    for word in sentence.split():
        stem = stemmer.stem(word)
        stemSentence += stem
        stemSentence += " "
    stemSentence = stemSentence.strip()
    return stemSentence
data['text'] = data['text'].apply(stemming)

In [41]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df5, random_state=42, test_size=0.20, shuffle=True)

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(strip_accents='unicode', analyzer='word', ngram_range=(1,3), norm='l2')
vectorizer.fit(train.text)
vectorizer.fit(test.text)
x_train = vectorizer.transform(train.text)
y_train = train.drop(labels = ['b_id','text'], axis=1)
x_test = vectorizer.transform(test.text)
y_test = test.drop(labels = ['b_id','text'], axis=1)

In [65]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
# Using pipeline for applying logistic regression and one vs rest classifier
LogReg_pipeline = Pipeline([
                ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=-1)),
            ])
for category in categories:
    print('**Processing {} comments...**'.format(category))
    
    # Training logistic regression model on train data
    LogReg_pipeline.fit(x_train, train[category])
    
    # calculating test accuracy
    prediction = LogReg_pipeline.predict(x_test)
   
    print('Test accuracy is {}'.format(accuracy_score(test[category], prediction)))
    print("Classification report")
    print(classification_report(test[category],prediction))
    print("\n")

**Processing service comments...**
Test accuracy is 0.9315861130020422
Classification report
              precision    recall  f1-score   support

           0       0.92      1.00      0.96      4682
           1       1.00      0.66      0.80      1194

    accuracy                           0.93      5876
   macro avg       0.96      0.83      0.88      5876
weighted avg       0.94      0.93      0.93      5876



**Processing price comments...**
Test accuracy is 0.8994213750850919
Classification report
              precision    recall  f1-score   support

           0       0.89      1.00      0.94      4711
           1       0.99      0.50      0.66      1165

    accuracy                           0.90      5876
   macro avg       0.94      0.75      0.80      5876
weighted avg       0.91      0.90      0.89      5876



**Processing place comments...**
Test accuracy is 0.9183117767188563
Classification report
              precision    recall  f1-score   support

           0